## Configure Snowflake Hierarchy

In [2]:
SNOW_WH = "E2E_SNOW_MLOPS_WH"
SNOW_DB = "E2E_SNOW_MLOPS_DB_CG"
SNOW_SCHEMA = "MLOPS_SCHEMA"
SNOW_ROLE = "E2E_SNOW_MLOPS_ROLE"

In [ ]:
-- Verify correct role is applied
SELECT CURRENT_USER();
USE ROLE E2E_SNOW_MLOPS_ROLE;

## Python Imports

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import math
import pickle
from datetime import datetime
import streamlit as st
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, precision_score, recall_score

# Snowpark ML
import snowflake.ml.modeling.preprocessing as snowml
import snowflake.ml.modeling.metrics as SnowMetrics
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.tune import get_tuner_context
from snowflake.ml.modeling import tune
from entities import search_algorithm

#Snowflake feature store
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# Snowpark session
from snowflake.snowpark import DataFrame
from snowflake.snowpark.functions import col, to_timestamp, min, max, month, dayofweek, dayofyear, avg, date_add, sql_expr
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.types import StringType
from snowflake.snowpark import Window
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as SnowF

SEED = 101

In [ ]:
def snowdf_shape(snowdf: DataFrame) -> tuple[int, int]:
    """
    Helper function to compute a snowpark dataframe's shape, pandas style
    """
    return (snowdf.count(), len(snowdf.columns))

## Collect Training Data

In [ ]:
# Get active session from snowflake, store as 'session'
session = get_active_session()

# Pull table from session and store as Snow DF (not pandas df!)
session_df = session.table("HUB.ASSET.INSTRUMENT_RISK_RANKING")
session_df.show(5)

In [ ]:
# Init feature store instance
fs = FeatureStore(
    session=session,
    database=SNOW_DB,
    name=SNOW_SCHEMA,
    default_warehouse=SNOW_WH,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)
fs.list_entities()

## Prep Data and Create FeatureView

In [ ]:

# Remove unnecessary features
session_df = session_df.drop([
    "CALENDAR_DATE",
    "RISK_RANK_REASON_TEXT",
    "IS_SELECT",
    "NOT_IN_SELECT_REASON",
    "PRICE_RISK_RANK",
    "SPGLOBAL_RATING_RISK_RANK",
    "SPGLOBAL_RATING_WATCHLIST_RISK_RANK",
    "MOODYS_RATING_RISK_RANK",
    "MOODYS_RATING_WATCHLIST_RISK_RANK"
])

# DEBUG: take only first instance of IID, remove instances of "-1"
# session_df = session_df.drop_duplicates("INSTRUMENT_IDENTIFIER")
# session_df = session_df.filter(SnowF.col("RISK_RANK") > -1.0)
session_df.show(5)
print(f"Rows: {session_df.count()}, Cols: {len(session_df.columns)}")

In [ ]:
pd = session_df.collect()
len(pd)

In [ ]:
# Diagnose current dataset
session_df.to_pandas()["RISK_RANK"].value_counts()
session_df.group_by("RISK_RANK").count().order_by("count")

In [ ]:
cleaned_df = session_df.filter(session_df["RISK_RANK"] >= SnowF.lit(0))
cleaned_df.group_by("RISK_RANK").count().order_by("count")

Note, the classifiers are very heavily skewed towards a Risk Rank of 3, with only one occurence of "-1".
To remedy this, we will drop the "-1" classifier (only appears for rows with missing data, which will be removed anyways), and randomly select the remaining classes

In [ ]:
# Create entity to track, joined on instrument id
# A bit of a necessary evil, but we can drop the extra col later on
new_entity = Entity(
    name="INSTRUMENT_ENTITY",
    join_keys=["INSTRUMENT_IDENTIFIER"],
    desc="Features joined per IID"
)

fs.register_entity(new_entity)

In [ ]:
# Add view for features, containing our tracked entity
fv = FeatureView(
    name="Risk_Rank_Feature_View",
    entities=[new_entity],
    feature_df=session_df
)

fv = fs.register_feature_view(fv, "v1", overwrite=True)
fs.list_feature_views()

In [ ]:
# Creates link for feature view UI
org = session.call("CURRENT_ORGANIZATION_NAME")
acc = session.call("CURRENT_ACCOUNT_NAME")
st.write(f"https://app.snowflake.com/{org}/{acc}/#/features/database/{SNOW_DB}/store/{SNOW_SCHEMA}")

## Retrieve Data from Feature View, Prep for Training

In [ ]:
# Snowpark dataset hook
dataset = fs.generate_dataset(
    name="RISK_RANK_DATASET",
    spine_df=session_df.select("INSTRUMENT_IDENTIFIER"),
    features=[fv],
    spine_label_cols=["RISK_RANK"]
)

In [ ]:
# Recast to snowpark DataFrame
dataset: DataFrame = dataset.read.to_snowpark_dataframe()
dataset.show(5)
dataset.print_schema()

In [ ]:
# Diagnose class distribution
y_dist = dataset.to_pandas()
y_dist["RISK_RANK"].value_counts()

# type(y_dist)

In [ ]:
# Categorize RISK_RANK classifiers
encoder = snowml.OrdinalEncoder(input_cols=["RISK_RANK"], output_cols=["RISK_RANK"], drop_input_cols=True)
dataset = encoder.fit(dataset).transform(dataset)

In [ ]:
# Split features and labels, send to pd (model friendly)
train, test = dataset.random_split(weights=[0.7, 0.3], seed=SEED)
x_train = train.drop("RISK_RANK", "INSTRUMENT_IDENTIFIER").to_pandas()
y_train = train[["RISK_RANK"]].to_pandas()
x_test = test.drop("RISK_RANK", "INSTRUMENT_IDENTIFIER").to_pandas()
y_test = test[["RISK_RANK"]].to_pandas()

x_train.shape
y_train.shape
x_test.shape
y_test.shape

## Train and Test Model

In [ ]:
# Init model of choice
xgb_model = XGBClassifier(
    max_depth=5,
    n_estimators=1,
    learning_rate=1e-3,
    booster="gbtree",
    seed=SEED
)

In [ ]:
# Train model
xgb_model.fit(x_train, y_train)

In [ ]:
# Measure training performance
train_preds = xgb_model.predict(x_train)
train_acc = sklearn.metrics.accuracy_score(y_train.to_numpy().squeeze(), train_preds)

print(f"Training Accuracy: {train_acc:.4f}")

In [ ]:
# Measure testing performance
test_preds = xgb_model.predict(x_test)
test_acc = sklearn.metrics.accuracy_score(y_test.to_numpy().squeeze(), test_preds)

print(f"Test Accuracy: {test_acc:.4f}")

## Diagnose Performance

In [ ]:
# Plot confusion matrix for training result (should be test, resampling needed)
conf_plot = sklearn.metrics.ConfusionMatrixDisplay.from_predictions(
    y_train.to_numpy().squeeze(),
    train_preds,
    display_labels=["-1", "0", "1", "2", "3"]
)

## Model Registry